# Identify Fraud from Enron Email
In this project, you will play detective, and put your machine learning skills to use by building an algorithm to identify Enron Employees who may have committed fraud based on the public Enron financial and email dataset.

## Section 1
* Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]

Enron Corporation was an American energy, commodities, and services company based in Houston, Texas. [https://en.wikipedia.org/wiki/Enron] Enron employed approximately 20,000 staff and was one of the world's major electricity, natural gas, communications and pulp and paper companies, with claimed revenues of nearly $101 billion during 2000. At the end of 2001, it was revealed that its reported financial condition was sustained by institutionalized, systematic, and creatively planned accounting fraud, known since as the Enron scandal. Enron has since become a well-known example of willful corporate fraud and corruption.

The goal of this project is to identify persons of interest (poi), which consist of:
* Indicted persons
* Reached a settlement or plea deal with the government
* Testified in exchange for prosecution immunity

Machine learning is an advance tool for classification tasks such as the Enron project. We can recognize patterns and regularities in our data and then make predictions or decisions, through the building of models from the sample inputs.

### Enron dataset cleaning and EDA

In [ ]:
import sys
import pickle
sys.path.append("./tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi',
                 'bonus',
                 'deferral_payments',
                 'deferred_income',
                 'director_fees',
                 'exercised_stock_options',
                 'expenses',
                 'from_messages',
                 'from_poi_to_this_person',
                 'from_this_person_to_poi',
                 'loan_advances',
                 'long_term_incentive',
                 'other',
                 'restricted_stock',
                 'restricted_stock_deferred',
                 'salary',
                 'shared_receipt_with_poi',
                 'to_messages',
                 'total_payments',
                 'total_stock_value',
                 'fraction_from_poi',
                 'fraction_to_poi']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

### Investigate dataset
Load data into a pandas dataframe for easier exploration.

In [ ]:
import pandas as pd

enron_df = pd.DataFrame.from_dict(data_dict, orient='index')
print(enron_df.shape)
print(enron_df.dtypes)
enron_df.head(n=10)

### Reformat 'NaN' to numpy nan
There are many 'NaN' values which should be reformated to numpy nan and then check again the data types of the Enron dataset.

In [ ]:
import numpy as np

# Convert NaN to numpy nan
enron_df.replace(to_replace='NaN', value=np.nan, inplace=True)

print("The sum of the nan values: \n{}".format(enron_df.isnull().sum()))
enron_df.dtypes

The missing values of the financial features are considered zero values and will be replaced later with 0s. As it was noted in the post:

https://discussions.udacity.com/t/how-to-start-the-final-project/177617/6

the financial feature values were extracted from the "enron61702insiderpay.pdf" document. Cells in the table marked as "-" are parsed as NaN values in the saved data dictionary, but actually represent values of 0. The conversion of NaNs to 0s is performed when the data is converted from a dictionary to a matrix through the featureFormat() function.

However, same does not apply for the email features. Missing values are probably reflective of a true lack of information about the person in the email corpus. Thus, considerations on treatment of email features based on missing values should be different than the treatment of financial features. For the simplicity of the project I will leave them as it is.

### Identify the higher salary
The bigger salary is the 'TOTAL', which is the sum of all the salaries of the listed Enron employees, so it is definetely an outlier and it will be removed.

In [ ]:
print(enron_df['salary'].idxmax(), enron_df['salary'].max())

### POIs

In [ ]:
total = enron_df['poi'].count()
pois = enron_df[enron_df['poi'] == True]['poi'].count()
print("POIs: ", pois)
print("Non POIs: ", total - pois)

### Enron employees exploration

In [ ]:
enron_df.index.values

- Let's find registries which have no non NAN values (recommended by the reviewer)

In [ ]:
for employee, nan_values in enron_df.isnull().sum(axis=1).items():
    if nan_values > 19:
        print(employee, nan_values)

- 'LOCKHART EUGENE E' contains only NaN data and it should be removed.

In [ ]:
enron_df.loc['LOCKHART EUGENE E']

### Remove outliers
The higher salary of 'TOTAL', as well as 'THE TRAVEL AGENCY IN THE PARK', which was revealed from the previous exploration of the list of Enron Employees will be removed, since they don't belong to real persons. 'LOCKHART EUGENE E' will be also removed as it doesn't contain any data. The graphical representation of bonus vs salary depicts some really high values but not unreasonable to pop them out.

In [ ]:
%pylab inline

enron_df.drop(['TOTAL', 'THE TRAVEL AGENCY IN THE PARK', 'LOCKHART EUGENE E'], inplace=True)
enron_df.plot.scatter(x = 'salary', y = 'bonus')

### Remove outliers from data_dict

In [ ]:
### Task 2: Remove outliers
data_dict.pop('TOTAL')
data_dict.pop('THE TRAVEL AGENCY IN THE PARK')
data_dict.pop('LOCKHART EUGENE E')


### Visualise the data
Visualising the data will help to understand our data and find any other outliers. There is a nice representation of the Enron data in the following link:

https://public.tableau.com/profile/diego2420#!/vizhome/Udacity/UdacityDashboard

This dashboard was made by Diego using Tableau and was posted in the Udacity Discussions Forum:

https://discussions.udacity.com/t/enron-data-set-visualization/33340

### Enron dataset conclusions
It appears that:
* There are 21 features
    * 1 "poi" (person of interest) feature
    * 6 email features
    * 14 financial features
* There are 146 Enron employees
    * 128 non POIs
    * 18 POIs
* All the features have missing 'NaN' values except from 'poi' which is a boolean.
* Exploring the data, it was found two features, 'TOTAL' and 'THE TRAVEL AGENCY IN THE PARK', which are not persons and was removed.
* Some extreme values which belong to persons are not considered outliers and will help the algorithms to identify pois.
* The dataset is susceptible to overfitting since it contains too many features and few data. In the following steps, we will try to apply various techniques to overcome this problem, such as:
    * Feature removal
    * Feature selection
    * Cross-validation

Missing values, i.e. 'NaN' strings will be replaced later with 0.0 using the featureFormat() function.

## Section 2
* What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “intelligently select features”, “properly scale features”]

### Create new features 
The number of sent/received emails to/from POIs is an interesting feature to be engineered and investigated. A more robust approach is probably the fraction of these emails to all emails than the absolute number of "poi emails". This ratio is the messages from/to POIs divided by to/from messages from the employee. In other words, this is a measure of frequency that an employee contacts with POIs. We expect a POI to send/receive emails with other POIs more frequently, hence the higher the ratio the more possible the person to be a POI.
1. fraction_to_poi
2. fraction_from_poi

In [ ]:
### Task 3: Create new feature(s)
# Create a function that computes either quantity
def computeFraction( poi_messages, all_messages ):
    ### take care of "NaN" when there is no known email address (and so
    ### no filled email features), and integer division.
    ### in case of poi_messages or all_messages having "NaN" value, return 0.
    fraction = 0
    if poi_messages == "NaN" or all_messages == "NaN":
        return fraction
    else:
        fraction = float(poi_messages)/float(all_messages)
        
    return fraction

for name in data_dict:

    data_point = data_dict[name]

    from_poi_to_this_person = data_point["from_poi_to_this_person"]
    to_messages = data_point["to_messages"]
    fraction_from_poi = computeFraction( from_poi_to_this_person, to_messages )
    data_point["fraction_from_poi"] = fraction_from_poi

    from_this_person_to_poi = data_point["from_this_person_to_poi"]
    from_messages = data_point["from_messages"]
    fraction_to_poi = computeFraction( from_this_person_to_poi, from_messages )
    data_point["fraction_to_poi"] = fraction_to_poi

### Select features
SelectKBest was used as a univariate feature selection which works by selecting the best features based on univariate statistical tests. SelectKBest removes all but the k highest scoring features.

In [ ]:
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

from sklearn.feature_selection import SelectKBest

select = SelectKBest()
select.fit(features, labels)
scores = select.scores_
# Show the scores in a table
feature_scores = zip(features_list[1:], scores)
ordered_feature_scores = sorted(feature_scores, key=lambda x: x[1], reverse=True)
for feature, score in ordered_feature_scores:
    print(feature, score)

* The table above shows all the features with their respective scores. The engineered feature "fraction_to_poi" is one of the most important features taking the fifth place, behind the "exercised_stock_options", "total_stock_value", "bonus" and "salary".
* SelectKBest process will be further examined by grid search in a pipeline. Instead of examining the features manually, a grid search of 'k' values in a pipeline will find the optimum 'k' values, which yield better algorithm performances. Numbers of 'k' ranged between 1 and 10, F1 and recall score were chosen as the evaluation metrics.

### Scale features
The next essential preprocessing step after feature engineering and feature selection is to scale all the features applying the "MinMaxScaler" method. Since the range of values of our data varies widely (several orders of magnitude), the machine learning algorithm functions will not work properly without normalization. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected. Therefore, the range of all features should be normalized so that each feature contributes approximately proportionately.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

## Section 3-6
* What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]
* What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? What parameters did you tune? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric items: “discuss parameter tuning”, “tune the algorithm”]
* What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric items: “discuss validation”, “validation strategy”]
* Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]

### Test algorithms
After creating new features, selecting the best features and scaling them, the next natural step is to run the algorithms to see how they perform after these preprocess operations. A useful scikit-learn tool, "Pipeline", packages the transformation steps of these operations with the estimation step of an algorithm classifier into a coherent workflow. The reasons to use "Pipeline" instead of keeping the steps separate are [https://www.civisanalytics.com/blog/workflows-in-python-using-pipeline-and-gridsearchcv-for-more-compact-and-comprehensive-code/]:

1. It makes code more readable (or, if you like, it makes the intent of the code clearer).
2. We don’t have to worry about keeping track data during intermediate steps, for example between transforming and estimating.
3. It makes it trivial to move ordering of the pipeline pieces, or to swap pieces in and out.
4. It allows you to do GridSearchCV on your workflow.

### Tune the algorithms
Tuning an algorithm is a process in which we optimize the parameters that impact the model in order to enable the algorithm to perform with an improved performance. If we don't tune the algorithms well, performance will be poor with low accuracy, precision or recall. Most of the machine learning algorithms contain a set of parameters (hyperparameters) which should be set up adequately to perform the best. While all of the algorithms attempt to set reasonable default hyperparameters for us, they can often fail to provide optimal results for many real world datasets in practice. To find an optimized combination of hyperparameters, a metric is chosen to measure the algorithm's performance on an independent data set and hyperparameters that maximize this measure are adopted. To make it more concrete through an example, the SVC classifier have several tunable hyperparameters (C, gamma, kernel, etc.) that can greatly affect the model fit. Their values can range greatly. For example C and gamma can be a float, while kernel can take ‘linear’, ‘poly’, ‘rbf’, or ‘sigmoid’ type. Our job is to specify the best combination of C, gamma values and kernel type, that produces optimal performances.

Tuning the models is a tedious, time-consuming process and there can sometimes be interactions between the choices we make in one step and the optimal value for a downstream step. Hopefully, there are two simple and easy tuning strategies, grid search and random search. Scikit-learn provides these two methods for algorithm parameter tuning. GridSearchCV allows us to construct a grid of all the combinations of parameters passing one classifier to pipeline each time, tries each combination, and then reports back the best combination. So, instead of trying numerous values for each tuning parameter, GridSearchCV will apply all the combinations of parameters - not just vary them independently - avoiding local optima.

To summarize, the power of GridSearchCV is that it multiplies out all the combinations of parameters and tries each one, making a k-fold cross-validated model for each combination. Then, we can ask for predictions and parameters from our GridSearchCV object and it will automatically return to us the best set of predictions, as well as the best parameters.

The following algoriths were studied:
* Naive Bayes
* Support Vector Machines
* Decision Trees
* K Nearest Neighbors
* Random Forest
* AdaBoost

### Validate analysis
The process of measuring the effectiveness of an algorithm for every possible input or how well it is fit and it generalizes to new data is called validation. A common mistake is to train and test the algorithm on the same data, which results to over-fitting, i.e. algorithm performs great on the training dataset, but it suffers on new data. To overcome this, data are separated into training and testing tests. In our analysis two validation steps applied through pipeline:
1. Data was separated into 70% training and 30% testing sets.
2. Because of the small size of the Enron dataset, stratified shuffle split method was used, which returns multiple stratified randomized splits, thus enhancing cross validation performance. A common problem with small datasets or with datasets that may have a class which dominates the others (in our study case, 88% of the data belongs to non POIs class) is the imbalance in the data. For example, when we split this dataset to 70% of training data, the training set may contain only one class of response variable, i.e. the non POIs (the majority class). To address this problem, stratified shuffle split divides randomly the data into multiple trials. For the investigation of the algorithms, 10 trials were created with 30% test set size. The same split ratio is used in tester.py but with 1000 folds.

### Evaluate the algorithms
Accuracy, i.e. the fraction of correct predictions is typically not enough information to evaluate a model. Although it is a starting point, it can lead to invalid decisions. Models with high accuracy may have inadequate precision or recall scores. For this reason the evaluation metrics that were assessed are:
1. Precision, the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative. The best value is 1 and the worst value is 0. In our study case, precision is when the algorithm guesses that somebody is a POI and actually measures how certain we are that this person really is a POI. For example, a precision of 0.3 means that if the model predicts 100 POIs, the 30 persons are POIs and the rest 70 are non POIs.
2. Recall, the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples. The best value is 1 and the worst value is 0. In context to the project, recall shows how well our identifier can pick the POIs. For example, a low recall score of 0.2 indicates that our identifier finds only 20% of all the real POIs in the prediction. The rest 80% of real POIs will not be investigated by the authorities if they trust our model, something that we don't want to happen.
3. F1 score,  a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is: F1 = 2 x (precision x recall) / (precision + recall).

In [ ]:
import warnings; warnings.simplefilter('ignore')
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import classification_report
from time import time

# Store to my_dataset for easy export below.
my_dataset = data_dict

# Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

# Instantiate the pipeline steps
select = SelectKBest()
nb = GaussianNB()
svc = SVC()
dtc = DecisionTreeClassifier()
knc = KNeighborsClassifier()
rfc = RandomForestClassifier()
abc = AdaBoostClassifier()

# Make a dictionary of classifiers
classifiers = {"GaussianNB": nb, "SVM": svc, "Decision Trees": dtc, 
               "KNN": knc, "Random Forest": rfc, "AdaBoost": abc}

# Create a function that combines pipeline and grid search and returns the best clf with the best parameters
def optimize_clf(clf, param_grid, n_splits):
    t0 = time()
    # Add pipeline steps into a list
    steps = [('feature_selection', select),
             ('clf', clf)]
    
    # Create the pipeline
    pipeline = Pipeline(steps)
    
    # Split data to train/test sets.
    features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)
    
    # Create Stratified ShuffleSplit cross-validator.
    # Provides train/test indices to split data in train/test sets.
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.3, random_state=42)
    
    # Create grid search
    gs = GridSearchCV(pipeline, param_grid, scoring=('f1', 'recall'),
                      cv=sss, refit='f1')
                
    # Fit pipeline on features_train and labels_train
    gs.fit(features_train, labels_train)
    # Call pipeline.predict() on features_test data to make a set of test predictions
    labels_pred = gs.predict(features_test)
    # Test predictions using sklearn.classification_report()
    report = classification_report(labels_test, labels_pred)
    # Find the best parameters and scores
    best_params = gs.best_params_
    best_score = gs.best_score_
    # Print the reports
    print("Report:")
    print(report)
    print("Best f1-score:")
    print(best_score)
    print("Best parameters:")
    print(best_params)
    print("Time passed: ", round(time() - t0, 3), "s")
    # Return the best estimator
    return gs.best_estimator_

for name, clf in classifiers.items():
    print("##########################################################################################################")
    print(name)
    if clf == nb:
        parameters = {'feature_selection__k':list(range(1, 10))}
    elif clf == svc:
        parameters = [{'feature_selection__k':list(range(1, 10)),
                      'clf__C':[10, 100]}]
    elif clf == dtc:
        parameters = [{'feature_selection__k':list(range(1, 10)),
                      'clf__criterion':['gini', 'entropy']}]
    elif clf == knc:
        parameters = [{'feature_selection__k':list(range(1, 10)),
                      'clf__n_neighbors':[3, 5, 7]}]
    elif clf == rfc:
        parameters = [{'feature_selection__k':list(range(1, 10)),
                      'clf__n_estimators':[1, 5, 10]}]
    elif clf == abc:
        parameters = [{'feature_selection__k':list(range(1, 10)),
                      'clf__n_estimators':[45, 50, 55]}]
    optimize_clf(clf, parameters, n_splits=10)

### Fine tuning
* After testing and tuning the algorithms, most of them perfoms quite well. The best algorithm in terms of acceptable f1 score and reasonable time cost was found to be the decision trees. AdaBoost and Random Forest perfomed equally or better than Decision Trees, but the required processing time is considerably greater. This is an important factor to consider when we want to run multiple times our algorithms. For this reason, decision trees will be further tuned.
* In most cases, 'k = 1' gives the best scores. Probably, that shows our algorithm is trained better with fewer features in such a small dataset. To improve algorithm learning ability, most of the features was finally removed and only the best 5 were examined.

## Results and Discussion

The table below shows the results after training and testing the decision tree algorithm without (first number) and with (second number) the created feature. In most of the cases, it seems that the new engineered feature have a positive impact on the performance. Definitely, they don't affect the performance negatively. Therefore, the algorithm will be further tested with the created features in the list.

| k | Accuracy           | Precision           | Recall            |
|------------------------|---------------------|-------------------|
|10 | 0.80013 / 0.81160  | 0.24696 / 0.28907   | 0.24350 / 0.28300 |
| 7 | 0.80380 / 0.81153  | 0.25658 / 0.28133   | 0.24850 / 0.26600 |
| 5 | 0.80660 / 0.80947  | 0.27687 / 0.27749   | 0.27950 / 0.26750 |
| 3 | 0.81327 / 0.81393  | 0.29130 / 0.30333   | 0.27950 / 0.30500 |
| 1 | 0.81973 / 0.81813  | 0.22713 / 0.22000   | 0.14650 / 0.14300 |

Finally, after several tests, a smaller list with five features worked better than the whole list of features. As it was mentioned previously, algorithms trained on small datasets with many features tend to overfit during training. Thus, they cannot generalise in new data and their performance is poor. The features included in the list were selected by their scores from the SelectKBest method.

Eventually, the model with the optimal set of features and parameters is:

### Decision Trees Classifier:

* Best parameters:
    * 'class_weight': None
    * 'criterion': 'gini'
    * 'max_features': 'auto'
    * 'splitter': 'best'
    * 'feature_selection_k': 5


* Feature Ranking:
    1. feature no. 1: total_stock_value (0.4318449749484233)
    2. feature no. 2: salary (0.19978632478632477)
    3. feature no. 3: fraction_to_poi (0.16519918459573657)
    4. feature no. 4: bonus (0.12090455840455819)
    5. feature no. 5: exercised_stock_options (0.08226495726495725)

The engineered features 'fraction_to_poi' was selected by SelectKBest method and the optimal 'k' value was found to be 5.

* Report generated by tester.py':

Pipeline(memory=None,
     steps=[('feature_selection', SelectKBest(k=5, score_func=<function f_classif at 0x111207e18>)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])
	Accuracy: 0.80571	Precision: 0.33349	Recall: 0.36050	F1: 0.34647	F2: 0.35475
	Total predictions: 14000	True positives:  721	False positives: 1441	False negatives: 1279	True negatives: 10559
